### Implementing Query Re-writing strategy (Sub-query decomposition) + Retrieval Methodology (Hypothetical Document Embeddings - HyDE)
In this section, we aim to **improve document retrieval capabilities** of our RAG system (Research Assistant v2) through (1) Sub-query decomposition and (2) HyDE methodology. More information of these methodologies below. Future works would include RAFT (retrieval augmented fine-tuning) to tackle the answer generation phase of our answer generator LLM. 

**Query re-writing and HyDE**
Under the context of query re-writing to optimise query retrieval and answer quality in RAG systems, Hypothetical Document Embeddings (HyDE) and Query Decomposition are techniques that tackle different fronts with an external LLM. HyDE aims to improve retrieval “accuracy” through generated and embedded “hypothetical documents” to answer user queries. These hypothetical documents are then used in retrieving real documents from the vector store via doc-doc similarity search to produce more “relevant” matches.  

Query Decomposition aims to improve answer quality from complex queries. The process breaks down complex queries/questions into smaller sub-questions/problems, which can either be solved sequentially (use first answer + retrieval for 2nd to answer 2nd question) or in parallel (consolidate each answer separately to form final answer).  

Other query transformation techniques to tackle various “human-centric” limitations with user queries include multi-query (to tackle query ambiguity) and step-back (when a higher-level conceptual understanding is required for accurate retrieval).

In [ ]:
# set autoreload for modules
%load_ext autoreload
%autoreload 2

# import dependencies
import os
import openai
from dotenv import load_dotenv, find_dotenv
import warnings
import nest_asyncio

_ = load_dotenv(find_dotenv())
# TO RESOLVE - Pull API key from OS into Jupyter notebook~
os.environ["OPENAI_API_KEY"] = "sk-.."
warnings.filterwarnings("ignore")
nest_asyncio.apply()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**First and foremost, let's import and format our evaluation dataset**

In [2]:
import json
import pandas as pd
from llama_index.core.llama_dataset import (
    LabelledRagDataset,
    LabelledRagDataExample,
    CreatedBy,
)


def get_rag_dataset_from_csv(csv_path: str):
    converters = {
        "reference_contexts":    lambda s: json.loads(s),
        "query_by":             lambda s: CreatedBy.model_validate_json(s),
        "reference_answer_by":  lambda s: CreatedBy.model_validate_json(s),
    }
    df = pd.read_csv(csv_path, converters=converters)
    examples = []
    for _, row in df.iterrows():
        examples.append(
            LabelledRagDataExample(
                query=row["query"],
                query_by=row["query_by"],                      # now a CreatedBy
                reference_contexts=row["reference_contexts"],   # now a List[str]
                reference_answer=row["reference_answer"],
                reference_answer_by=row["reference_answer_by"], # now a CreatedBy
            )
        )
    # Create the dataset
    dataset = LabelledRagDataset(examples=examples)
    return dataset

In [3]:
eval_dataset = get_rag_dataset_from_csv("data/eval_dataset.csv")
len(eval_dataset.examples)

55

**Next, we re-build the query engine (our RAG System)**

In [4]:
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.llms.openai import OpenAI

# Configure LLM
Settings.llm = OpenAI(model="gpt-4o-mini")

# Use custom embedding model - “hkunlp/instructor-large”
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# load embedding model (try) - loads https://huggingface.co/hkunlp/instructor-large
Settings.embed_model = HuggingFaceEmbedding(model_name="hkunlp/instructor-large")

In [5]:
from llama_index.llms.ollama import Ollama

# Instantiate query engine LLM - Set timeout to ___s to allow sufficient time for answer generation
llm = Ollama(model="llama3.2:1b", request_timeout=3000)

In [6]:
# Input documents (in index), embedding model and LLM to generate query engine (RAG system)
docs = SimpleDirectoryReader("../RAG-webscraper/docs/").load_data(show_progress=True)
index = VectorStoreIndex.from_documents(docs, embed_model=Settings.embed_model)
query_engine = index.as_query_engine(similarity_top_k=6, llm=llm)

Loading files:  50%|█████     | 1/2 [00:00<00:00,  1.49it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Loading files: 100%|██████████| 2/2 [00:00<00:00,  2.61it/s]


#### 1. Building & Evaluating our Sub-Question Query Engine
**Let us first incorporate "sub-query decomposition" to our RAG system**

The sub-question query engine first breaks down the complex query into sub-questions, retrieves relevant documents from the data source, then gathers all the intermediate responses and finally, synthesizes a final response! 

In [7]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="docs",
            description="Project reports",
        ),
    ),
]

# wrap query engine tool in Sub-Question Query Engine object - Final Query Engine
query_engine_final = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [8]:
# Test response
response = query_engine_final.query(
    "What is the primary focus of the project described in the report, and how does it aim to address the challenges of content moderation on social media platforms?"
)

Retrying llama_index.llms.openai.base.OpenAI._chat in 1.0 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._chat in 1.287152439880236 seconds as it raised APIConnectionError: Connection error..


APIConnectionError: Connection error.

In [10]:
print(response)

The primary focus of the project is on evaluating a data collection and project dataset for an AI or machine learning system, with an emphasis on various aspects such as data sources, labeling, statistics, imbalances, formats, privacy, and ethical considerations. It also encompasses the stages and architecture of a machine learning model from building to deployment, aiming to develop a robust model for decision-making or prediction.

To address the challenges of content moderation on social media platforms, the project proposes several strategies. These include monitoring and reducing model prediction latency, implementing robust verification processes to mitigate adversarial attacks, and utilizing a Canary deployment strategy for testing new model versions. Additionally, it emphasizes ethical and legal considerations in data handling, expands the project scope to include various types of harmful content, and incorporates a monitoring and retraining step to adapt to real-world conditio

**Food for thought:** *Sub-question query engine generated very detailed and granular responses as compared to ground-truth answers.*

*Response seems accurate!* **Let us now evaluate the "re-vamped sub-question query engine (after implementation of sub-question query decomposition)**

In [14]:
from llama_index.core.llama_pack import download_llama_pack

RagEvaluatorPack = download_llama_pack("RagEvaluatorPack", "./pack")

# Instantiate RAG Evaluator - input query engine, evaluation dataset, judge LLM & embeddings model
rag_evaluator = RagEvaluatorPack(
    query_engine=query_engine_final, 
    rag_dataset=eval_dataset,
    judge_llm=Settings.llm, #use the same llm that we use to create the dataset to judge
    embed_model=Settings.embed_model
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing /Users/jinkettyee/Desktop/my_GitHub/great-things/RAG-evaluation/pack
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for llama-index-packs-rag-evaluator: filename=llama_index_packs_rag_evaluator-0.3.1-py3-none-any.whl size=4935 sha256=dad027ca78706e90f46566970ee58c33a8d05a5fec0ef37e09c13ed24e871a8c
  Stored in directory: /private/var/folders/nr/6b6zx3jn687ghmtz2_2dw_b40000gn/T/pip-ephem-wheel-cache-g707gl3y/wheels/c5/b3/f2/e8724b5fcdbbb7cd612ab30b90396f09760893fb336af49b78
Successfully built llama-index-packs-rag-evaluator
  Attempting uninstall: llama-index-packs-rag-evaluator
    Found existing installation: llama-index-packs-rag-evaluator 0.3.1
    Uninstalling llama-index-packs-

You should consider upgrading via the '/Users/jinkettyee/.pyenv/versions/great_things/bin/python -m pip install --upgrade pip' command.


In [15]:
# Run in async mode
nest_asyncio.apply()

# run evaluator function
benchmark_df = await rag_evaluator.arun()

Batch processing of predictions:   0%|          | 0/10 [00:00<?, ?it/s]

Generated 2 sub questions.
[docs] Q: What specific method did the team use for data labeling in their explicit image classification project?
[docs] Q: What advantages were identified by the team regarding the data labeling method used in the explicit image classification project?
Generated 2 sub questions.
[docs] Q: What are the main components of the machine learning system architecture described in the document?
[docs] Q: What is the purpose of each component in the machine learning system architecture as described in the document?
Generated 3 sub questions.
[docs] Q: What are the details of Group 9 in the eyecatcher project report?
[docs] Q: Can you provide the list of members in Group 9 from the eyecatcher project report?
[docs] Q: What roles do the members of Group 9 play in the eyecatcher project report?
Generated 3 sub questions.
[docs] Q: What is the primary focus of the project described in the report?
[docs] Q: What challenges of content moderation on social media platforms a

Batch processing of predictions:  10%|█         | 1/10 [09:17<1:23:37, 557.50s/it]

[docs] A: Amazon Rekognition uses an existing trained model for classification in its "DetectModerationLabels" method.
[docs] A: Implementing a machine learning tool for explicit image classification in social media platforms would likely have various cost implications. Here are some potential considerations:

1. **Data Collection and Labeling**: The project aims to use Amazon Rekognition as the data labeling solution, which may incur costs associated with accessing and utilizing Rekognition services. Additionally, collecting user feedback for concept drift detection and model drift monitoring could require additional personnel or resources.
2. **Model Training and Deployment**: The team will need to train a machine learning model on a large dataset of labeled images, which can be resource-intensive. Deploying the trained model in a scalable environment might involve additional costs for infrastructure, computing resources, and potential cloud migration expenses.
3. **Cloud Infrastruct

Batch processing of predictions:  20%|██        | 2/10 [10:27<36:06, 270.86s/it]  

...

Batch processing of predictions:  60%|██████    | 3/5 [11:58<05:40, 170.07s/it]

[docs] A: Yes, there are several case studies or examples mentioned in the project reports that illustrate successful enhancement of training data quality for speech recognition. 

One such example is from the section titled "Training Report – Results, Evaluation and Future Works" on page 9, where it mentions:

"In this study, we compared our fine-tuned model's performance against a pre-trained baseline 'wav2vec2-large-960h' model development set (cv-valid-dev). Key dataset features and results are displayed in Table 1."

Table 1 shows the comparison of WER scores between the fine-tuned "wav2vec2-large-960h" model and the pre-trained "wav2vec2-large-960h" baseline model development set on different datasets.

In another section, titled "3. Limitations, Considerations & Future Works", on page 4, it mentions:

"One key limitation of this project is compute and memory limitations. We were only able to fine-tune our pre-trained 'wav2vec2-large-960h' model on 6,300 audio files."

This sugge

Batch processing of predictions:  80%|████████  | 4/5 [12:27<01:54, 114.21s/it]

[docs] A: Based on the provided context information, several methodologies have been proposed or explored for integrating Large Language Models (LLMs) into existing speech recognition systems. Some of these include:

1. **Semi-Supervised Learning Strategies**: Utilizing high-confidence transcriptions to supplement the training data pool and improve model fine-tuning.
2. **External Language Model Integration**: Leveraging pre-trained language models, such as transformer-based models like BERT or RoBERTa, to enhance speech recognition performance.
3. **Hybrid Approaches**: Combining pre-trained models with other components, like audio augmentation or domain adaptation strategies, to achieve better results.
4. **Data Augmentation Techniques**: Applying techniques like speech perturbations, time masking, pitch shift, and background noise injection to increase the diversity of training data.
5. **Domain Adaptation Strategies**: Using domain-specific models or transfer learning to adapt pre-

Batch processing of predictions: 100%|██████████| 5/5 [12:57<00:00, 155.45s/it]
Batch processing of evaluations:  32%|███▏      | 9/28.0 [00:53<02:28,  7.84s/it]Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-NrMfKyqsGd6aFUijloMM0e8A on tokens per min (TPM): Limit 200000, Used 200000, Requested 631. Please try again in 189ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-NrMfKyqsGd6aFUijloMM0e8A on tokens per min (TPM): Limit 200000, Used 200000, Requested 21854. Please try again in 6.556s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type'

In [17]:
print(benchmark_df)

rag                            base_rag
metrics                                
mean_correctness_score         3.600000
mean_relevancy_score           0.872727
mean_faithfulness_score        0.745455
mean_context_similarity_score  0.908821


#### 2. Building & Evaluating a HyDE Document Retrieval Query Engine
Great! Are there any improvements? **Now, let us evaluate the substitution of conventional similarity search with hypothetical document embedding (HyDE) search~**

Review potential benefits, limitations and "failure cases" of Hypothetical Document Embedding (HyDE) implementation from: https://docs.llamaindex.ai/en/stable/examples/query_transformations/HyDEQueryTransformDemo/

In [8]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display

query_str = "What is the primary focus of the project described in the report, and how does it aim to address the challenges of content moderation on social media platforms?"
# Use HyDEQueryTransform to generate hypothetical documents for improved document lookup from vector store. 
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>The primary focus of the project described in the report is to develop a deployable machine learning pipeline for explicit image classification, specifically for detecting and filtering out explicit nudity from images. The project aims to address the challenges of content moderation on social media platforms by developing a system that can accurately detect and classify explicit images.

To achieve this goal, the project involves several steps:

1. Data collection and preprocessing: The team collects and preprocesses a dataset of labeled images to improve the quality and accuracy of the model.
2. Model building: The team builds a machine learning model using Python, PyTorch, and SageMaker to classify explicit images.
3. Deployment: The trained model is deployed as an Amazon Lambda function, which can be triggered by user reports of explicit content.

The project addresses several challenges in content moderation on social media platforms, including:

1. Data quality and availability: The project aims to address the issue of limited labeled data for training models, which can lead to poor performance.
2. Data imbalance: The project targets a class imbalance problem, where some classes (e.g., explicit images) have a larger number of instances than others (e.g., safe images).
3. User engagement and feedback: The project includes features such as user feedback loops and appeals mechanisms to encourage users to report objectionable content.

By developing a robust and scalable machine learning system, the project aims to improve the accuracy and efficiency of explicit image classification on social media platforms, ultimately contributing to a safer and more responsible online environment.</b>

*Now, let's evaluate the effectiveness of HyDE search vs Document Similarity Search for Document Retrieval*

In [10]:
# Run in async mode
nest_asyncio.apply()

from llama_index.core.llama_pack import download_llama_pack
RagEvaluatorPack = download_llama_pack("RagEvaluatorPack", "./pack")

# Instantiate RAG Evaluator - input query engine, evaluation dataset, judge LLM & embeddings model
rag_evaluator = RagEvaluatorPack(
    query_engine=hyde_query_engine, 
    rag_dataset=eval_dataset,
    judge_llm=Settings.llm, #use the same llm that we use to create the dataset to judge
    embed_model=Settings.embed_model
)

benchmark_hyde = await rag_evaluator.arun()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing /Users/jinkettyee/Desktop/my_GitHub/great-things/RAG-evaluation/pack
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for llama-index-packs-rag-evaluator: filename=llama_index_packs_rag_evaluator-0.3.1-py3-none-any.whl size=4935 sha256=dad027ca78706e90f46566970ee58c33a8d05a5fec0ef37e09c13ed24e871a8c
  Stored in directory: /private/var/folders/nr/6b6zx3jn687ghmtz2_2dw_b40000gn/T/pip-ephem-wheel-cache-dfroggg1/wheels/c5/b3/f2/e8724b5fcdbbb7cd612ab30b90396f09760893fb336af49b78
Successfully built llama-index-packs-rag-evaluator
  Attempting uninstall: llama-index-packs-rag-evaluator
    Found existing installation: llama-index-packs-rag-evaluator 0.3.1
    Uninstalling llama-index-packs-

You should consider upgrading via the '/Users/jinkettyee/.pyenv/versions/great_things/bin/python -m pip install --upgrade pip' command.
Batch processing of predictions: 100%|██████████| 5/5 [02:43<00:00, 32.69s/it]
Batch processing of evaluations: 100%|██████████| 28/28.0 [01:55<00:00,  4.14s/it]


In [11]:
print(benchmark_hyde)

rag                            base_rag
metrics                                
mean_correctness_score         3.145455
mean_relevancy_score           0.872727
mean_faithfulness_score        0.636364
mean_context_similarity_score  0.953062


*Are there any improvements in evaluation scores (response & retrieval) from HyDE document retrieval?*